In [ ]:
from nbdev import show_doc

In [ ]:
#| default_exp core

# core

> Contains core components for Raspberry Pi Object Detection over the network

## Setup

Required environment variables in `.env` file

- `PI_PASSWORD`
    - to use `capture_and_fetch` on pi server
- `DISCORD_ID`
    - This is the Discord client ID token
- `DISCORD_TOKEN`
    - Discord bot token


## Camera and Network

> Handles connection to Pi, taking pictures, and saving image locally

In [ ]:
#| export
import os
FILE_PATH = os.path.abspath('') 

In [ ]:
#| eval: false
#| skip_showdoc
#| hideline
FILE_PATH = (globals()['_dh'][0] / 'odd_pi').as_posix()

In [ ]:
# | export
import random

import discord
from pathlib import Path
import polars as pl
import paramiko
import time
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
from dotenv import load_dotenv

DATA_DIR = Path(FILE_PATH).parent.parent / "data"
NAME_PATH = (DATA_DIR.parent / "models") / "yolo.names"
TEST_PATH = DATA_DIR / "test.jpg"
CMD = "raspistill -t 0 -h 640 -w 640 -o ~/Desktop/capture.jpg"


def pltimg(img: cv2.Mat) -> plt.Figure:
    """Plots picture"""
    return plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))


def capture_and_fetch(
    hostname: str,
    username: str = "pi",
    local_path: str = "./data/capture.jpg",
    delay: int = None,
    cmd=CMD,
):
    """Requires `PI_PASSWORD be set in .env file. Delay is the number of milliseconds before taking picture"""
    load_dotenv()
    password = os.environ["PI_PASSWORD"]
    ssh_client = _connect(hostname, username, password)

    if delay:
        time.sleep(delay)

    ssh_client.exec_command(cmd)
    time.sleep(0.5)

    _fetch(ssh_client, "/home/pi/Desktop/capture.jpg", local_path)


def _connect(hostname: str, username: str, password: str) -> paramiko.SSHClient():
    """Connects to client"""
    ssh_client = paramiko.SSHClient()
    ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh_client.connect(hostname=hostname, username=username, password=password)
    return ssh_client


def _fetch(client: paramiko.SSHClient, remote_path: str, local_path: str):
    """Fetches single file using SSHClient"""
    ftp_client = client.open_sftp()
    ftp_client.get(remote_path, local_path)
    ftp_client.close()



In [ ]:
show_doc(capture_and_fetch)

---

### capture_and_fetch

>      capture_and_fetch (hostname:str, username:str='pi',
>                         local_path:str='./data/capture.jpg', delay:int=None,
>                         cmd='raspistill -t 0 -h 640 -w 640 -o
>                         ~/Desktop/capture.jpg')

Requires `PI_PASSWORD be set in .env file. Delay is the number of milliseconds before taking picture

## Model and Prediction

> Fetch YOLOv5 model and predict on image

In [ ]:
#| export
from cvu.detector.yolov5 import Yolov5 as Yolov5Onnx

def predict(model, image_path):
    img = cv2.imread(image_path)
    preds = model(img)
    fig = pltimg(preds.draw(img))
    fig.write_png(image_path[:-4] + "_yolo.png")
    return preds, img


def get_default_model():
    return Yolov5Onnx(classes="coco", backend="onnx", weight="yolov5s", device="cpu")

In [ ]:
show_doc(predict)

---

### predict

>      predict (model, image_path)

Example use

```python
model = get_default_model()
preds, img = predict(model, '/path/to/img')
```

## Discord Bot

> Manage interactions to camera via Discord Bot

In [ ]:
# Required to run in jupyter but not in our package
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# | export

TOKEN = os.getenv("DISCORD_TOKEN")

intents = discord.Intents.default()
intents.message_content = True

client = discord.Client(intents=intents)


@client.event
async def on_ready():
    print(f"{client.user} has connected to Discord. Hello!")
    # for guild in client.guilds:
    #         print(f'{client.user} is connected to {guild.name}\tID: {guild.id}')


@client.event
async def on_member_join(member):
    await member.create_dm()
    await member.dm_channel.send(
        f"Hello {member.name}, welcome to Islander Walk Securitron 9000"
    )


@client.event
async def on_message(message):
    if message.author == client.user:
        return

    b99 = ["Bingpot", "Cool. Cool Cool Cool Cool", "no doubt no doubt no doubt"]
    if message.content == "99!":
        response = random.choice(b99)
        await message.channel.send(response)


def take_picture(message):
    capture_and_fetch()


# client.run(TOKEN)


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()